In [1]:
!pip install pyngrok

from pyngrok import ngrok, conf

In [2]:
ngrok.kill()

In [3]:
# Replace YOUR_NGROK_AUTH_TOKEN with your actual ngrok token
ngrok.set_auth_token('2mWGnibfBQXTw4MpKCZms5Lmxkf_53xvjcgP1kh2os73ZDJws')
# Start an ngrok tunnel for port 5000
# mlflow_url = ngrok.connect(5000, "http")
# print(f"MLflow Tracking UI available at: {mlflow_url}")

In [4]:
!pip install mlflow
!pip install kaggle
from google.colab import files
files.upload()  # Upload the kaggle.json file


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 555.1/555.1 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"mohamedahmed50","key":"03e5f770216f7c4027369e76659ef69c"}'}

In [5]:
!mkdir -p ~/.kaggle  # Create the .kaggle folder
!mv kaggle.json ~/.kaggle/  # Move the kaggle.json to the .kaggle folder
!chmod 600 ~/.kaggle/kaggle.json  # Set proper permissions


In [6]:
!kaggle competitions download -c microsoft-malware-prediction


100% 1.54G/1.54G [01:27<00:00, 21.6MB/s]
100% 1.54G/1.54G [01:27<00:00, 18.8MB/s]


In [7]:
!unzip microsoft-malware-prediction.zip


Archive:  microsoft-malware-prediction.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


# Import Lib

In [8]:
import pandas as pd
from scipy.stats import skew
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_selection import SelectKBest, chi2, f_classif, mutual_info_classif
import mlflow

# Read Data

In [9]:

# Read the first 500,000 rows from both training and test datasets
train_data = pd.read_csv('train.csv', nrows=500000)
test_data = pd.read_csv('test.csv', nrows=500000)

pd.set_option('display.max_columns', None)

# Display the first few rows of the training dataset
train_data.head()


<ipython-input-9-24c06b41218f>:2: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv('train.csv', nrows=500000)
<ipython-input-9-24c06b41218f>:3: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  test_data = pd.read_csv('test.csv', nrows=500000)


,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,CityIdentifier,OrganizationIdentifier,GeoNameIdentifier,LocaleEnglishNameIdentifier,Platform,Processor,OsVer,OsBuild,OsSuite,OsPlatformSubRelease,OsBuildLab,SkuEdition,IsProtected,AutoSampleOptIn,PuaMode,SMode,IeVerIdentifier,SmartScreen,Firewall,UacLuaenable,Census_MDC2FormFactor,Census_DeviceFamily,Census_OEMNameIdentifier,Census_OEMModelIdentifier,Census_ProcessorCoreCount,Census_ProcessorManufacturerIdentifier,Census_ProcessorModelIdentifier,Census_ProcessorClass,Census_PrimaryDiskTotalCapacity,Census_PrimaryDiskTypeName,Census_SystemVolumeTotalCapacity,Census_HasOpticalDiskDrive,Census_TotalPhysicalRAM,Census_ChassisTypeName,Census_InternalPrimaryDiagonalDisplaySizeInInches,Census_InternalPrimaryDisplayResolutionHorizontal,Census_InternalPrimaryDisplayResolutionVertical,Census_PowerPlatformRoleName,Census_InternalBatteryType,Census_InternalBatteryNumberOfCharges,Census_OSVersion,Census_OSArchitecture,Census_OSBranch,Census_OSBuildNumber,Census_OSBuildRevision,Census_OSEdition,Census_OSSkuName,Census_OSInstallTypeName,Census_OSInstallLanguageIdentifier,Census_OSUILocaleIdentifier,Census_OSWUAutoUpdateOptionsName,Census_IsPortableOperatingSystem,Census_GenuineStateName,Census_ActivationChannel,Census_IsFlightingInternal,Census_IsFlightsDisabled,Census_FlightRing,Census_ThresholdOptIn,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
0,0000028988387b115f69f31a3bf04f09,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1735.0,0,7.0,0,NaN,53447.0,1.0,1.0,1,29,128035.0,18.0,35.0,171,windows10,x64,10.0.0.0,17134,256,rs4,17134.1.amd64fre.rs4_release.180410-1804,Pro,1.0,0,NaN,0.0,137.0,NaN,1.0,1.0,Desktop,Windows.Desktop,2668.0,9124.0,4.0,5.0,2341.0,NaN,476940.0,HDD,299451.0,0,4096.0,Desktop,18.9,1440.0,900.0,Desktop,NaN,4.294967e+09,10.0.17134.165,amd64,rs4_release,17134,165,Professional,PROFESSIONAL,UUPUpgrade,26.0,119,UNKNOWN,0,IS_GENUINE,Retail,NaN,0.0,Retail,NaN,628.0,36144.0,0,NaN,0.0,0,0,0.0,0.0,10.0,0
1,000007535c3f730efa9ea0b7ef1bd645,win8defender,1.1.14600.4,4.13.17134.1,1.263.48.0,0,7.0,0,NaN,53447.0,1.0,1.0,1,93,1482.0,18.0,119.0,64,windows10,x64,10.0.0.0,17134,256,rs4,17134.1.amd64fre.rs4_release.180410-1804,Pro,1.0,0,NaN,0.0,137.0,NaN,1.0,1.0,Notebook,Windows.Desktop,2668.0,91656.0,4.0,5.0,2405.0,NaN,476940.0,HDD,102385.0,0,4096.0,Notebook,13.9,1366.0,768.0,Mobile,NaN,1.000000e+00,10.0.17134.1,amd64,rs4_release,17134,1,Professional,PROFESSIONAL,IBSClean,8.0,31,UNKNOWN,0,OFFLINE,Retail,NaN,0.0,NOT_SET,NaN,628.0,57858.0,0,NaN,0.0,0,0,0.0,0.0,8.0,0
2,000007905a28d863f6d0d597892cd692,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1341.0,0,7.0,0,NaN,53447.0,1.0,1.0,1,86,153579.0,18.0,64.0,49,windows10,x64,10.0.0.0,17134,768,rs4,17134.1.amd64fre.rs4_release.180410-1804,Home,1.0,0,NaN,0.0,137.0,RequireAdmin,1.0,1.0,Desktop,Windows.Desktop,4909.0,317701.0,4.0,5.0,1972.0,NaN,114473.0,SSD,113907.0,0,4096.0,Desktop,21.5,1920.0,1080.0,Desktop,NaN,4.294967e+09,10.0.17134.165,amd64,rs4_release,17134,165,Core,CORE,UUPUpgrade,7.0,30,FullAuto,0,IS_GENUINE,OEM:NONSLP,NaN,0.0,Retail,NaN,142.0,52682.0,0,NaN,0.0,0,0,0.0,0.0,3.0,0
3,00000b11598a75ea8ba1beea8459149f,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1527.0,0,7.0,0,NaN,53447.0,1.0,1.0,1,88,20710.0,NaN,117.0,115,windows10,x64,10.0.0.0,17134,256,rs4,17134.1.amd64fre.rs4_release.180410-1804,Pro,1.0,0,NaN,0.0,137.0,ExistsNotSet,1.0,1.0,Desktop,Windows.Desktop,1443.0,275890.0,4.0,5.0,2273.0,NaN,238475.0,UNKNOWN,227116.0,0,4096.0,MiniTower,18.5,1366.0,768.0,Desktop,NaN,4.294967e+09,10.0.17134.228,amd64,rs4_release,17134,228,Professional,PROFESSI

# Print Unique Values

In [10]:
# Print the number of unique values for each column in the dataset
for column in train_data.columns:
    num_unique_values = train_data[column].nunique()
    print(f"Column: {column}")
    print(f"Number of unique values: {num_unique_values}\n")


Column: MachineIdentifier
Number of unique values: 500000

Column: ProductName
Number of unique values: 5

Column: EngineVersion
Number of unique values: 55

Column: AppVersion
Number of unique values: 93

Column: AvSigVersion
Number of unique values: 6506

Column: IsBeta
Number of unique values: 2

Column: RtpStateBitfield
Number of unique values: 6

Column: IsSxsPassiveMode
Number of unique values: 2

Column: DefaultBrowsersIdentifier
Number of unique values: 557

Column: AVProductStatesIdentifier
Number of unique values: 5489

Column: AVProductsInstalled
Number of unique values: 5

Column: AVProductsEnabled
Number of unique values: 5

Column: HasTpm
Number of unique values: 2

Column: CountryIdentifier
Number of unique values: 222

Column: CityIdentifier
Number of unique values: 37307

Column: OrganizationIdentifier
Number of unique values: 43

Column: GeoNameIdentifier
Number of unique values: 267

Column: LocaleEnglishNameIdentifier
Number of unique values: 233

Column: Platform
N

# **Code for printing the percentage of repeatition**

In [11]:
# Define a threshold for "very much repeated" values
threshold = 0.5  # For example, values that occur more than 50% of the time

# Check for columns with a very much repeated number
for column in train_data.columns:
    value_counts = train_data[column].value_counts()
    most_common_value = value_counts.idxmax()
    most_common_count = value_counts.max()
    total_count = len(train_data[column])

    # Calculate the percentage of the most common value
    most_common_percentage = (most_common_count / total_count) * 100

    if most_common_percentage > (threshold * 100):
        print(f"Column: {column}")
        print(f"Most common value: {most_common_value}")
        print(f"Frequency: {most_common_count} ({most_common_percentage:.2f}%)\n")


Column: ProductName
Most common value: win8defender
Frequency: 494638 (98.93%)

Column: AppVersion
Most common value: 4.18.1807.18075
Frequency: 288625 (57.73%)

Column: IsBeta
Most common value: 0
Frequency: 499995 (100.00%)

Column: RtpStateBitfield
Most common value: 7.0
Frequency: 484819 (96.96%)

Column: IsSxsPassiveMode
Most common value: 0
Frequency: 491357 (98.27%)

Column: AVProductStatesIdentifier
Most common value: 53447.0
Frequency: 326795 (65.36%)

Column: AVProductsInstalled
Most common value: 1.0
Frequency: 348436 (69.69%)

Column: AVProductsEnabled
Most common value: 1.0
Frequency: 485030 (97.01%)

Column: HasTpm
Most common value: 1
Frequency: 493911 (98.78%)

Column: Platform
Most common value: windows10
Frequency: 482943 (96.59%)

Column: Processor
Most common value: x64
Frequency: 454511 (90.90%)

Column: OsVer
Most common value: 10.0.0.0
Frequency: 483720 (96.74%)

Column: OsSuite
Most common value: 768
Frequency: 311979 (62.40%)

Column: SkuEdition
Most common val

# print number of nulls

In [12]:
# Assuming train_data is your DataFrame
# train_data = pd.read_csv('train.csv')  # Uncomment if needed
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)  # Optional: to display all rows
# Print the number of NaN or null values in each column
null_counts = train_data.isnull().sum()
print("Number of NaN or null values in each column:")
print(null_counts)


Number of NaN or null values in each column:
MachineIdentifier                                         0
ProductName                                               0
EngineVersion                                             0
AppVersion                                                0
AvSigVersion                                              0
IsBeta                                                    0
RtpStateBitfield                                       1874
IsSxsPassiveMode                                          0
DefaultBrowsersIdentifier                            475698
AVProductStatesIdentifier                              2031
AVProductsInstalled                                    2031
AVProductsEnabled                                      2031
HasTpm                                                    0
CountryIdentifier                                         0
CityIdentifier                                        18209
OrganizationIdentifier                               15

# Drop Specific Columns

In [13]:
# Initialize an empty list for columns to drop
columns_to_drop = [
    "MachineIdentifier",
    "ProductName",
    "CountryIdentifier",
    "IeVerIdentifier",
    "Census_OEMNameIdentifier",
    "Census_OEMModelIdentifier",
    "Census_OSUILocaleIdentifier",
    "Census_FirmwareManufacturerIdentifier",
    "Census_FirmwareVersionIdentifier",
    "Wdft_RegionIdentifier",
    "IsBeta",
    "AVProductStatesIdentifier",
    "AutoSampleOptIn",
    "UacLuaenable",
    "Census_ProcessorManufacturerIdentifier",
    "Census_IsPortableOperatingSystem",
    "Census_GenuineStateName",
    "Census_IsFlightsDisabled",
    "Census_IsVirtualDevice",
    "PuaMode",
    "DefaultBrowsersIdentifier",
    "OrganizationIdentifier",
    "Census_ProcessorClass",
    "Census_InternalBatteryType",
    "Census_IsFlightingInternal",
    "Census_IsWIMBootEnabled",

]

# Example of adding column names to the list
# columns_to_drop.append('ColumnName1')
# columns_to_drop.append('ColumnName2')

# Drop the specified columns from the DataFrame
train_data_cleaned = train_data.drop(columns=columns_to_drop, errors='ignore')

# Fill missing values

In [14]:
# Function to separate numeric and categorical columns
def separate_numeric_categorical(df):
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    categorical_cols = df.select_dtypes(exclude=[np.number]).columns
    return numeric_cols, categorical_cols

# Function to impute missing values
def impute_missing_values(df, numeric_cols, categorical_cols):
    df_imputed = df.copy()

    # Impute numeric columns with the mean
    numeric_imputer = SimpleImputer(strategy='mean')
    df_imputed[numeric_cols] = numeric_imputer.fit_transform(df_imputed[numeric_cols])

    # Impute categorical columns with the mode (most frequent value)
    categorical_imputer = SimpleImputer(strategy='most_frequent')
    df_imputed[categorical_cols] = categorical_imputer.fit_transform(df_imputed[categorical_cols])

    return df_imputed

# Apply the functions
numeric_columns, categorical_columns = separate_numeric_categorical(train_data_cleaned)

# Impute missing values
train_data = impute_missing_values(train_data_cleaned, numeric_columns, categorical_columns)

# Check the result
print(train_data.isnull().sum())

EngineVersion                                        0
AppVersion                                           0
AvSigVersion                                         0
RtpStateBitfield                                     0
IsSxsPassiveMode                                     0
AVProductsInstalled                                  0
AVProductsEnabled                                    0
HasTpm                                               0
CityIdentifier                                       0
GeoNameIdentifier                                    0
LocaleEnglishNameIdentifier                          0
Platform                                             0
Processor                                            0
OsVer                                                0
OsBuild                                              0
OsSuite                                              0
OsPlatformSubRelease                                 0
OsBuildLab                                           0
SkuEdition

# Data Label Encoding

In [15]:

# Load your dataset
# train_data = pd.read_csv('train.csv')  # Uncomment if not already loaded

# Identify categorical columns
categorical_columns = train_data.select_dtypes(include=['object']).columns

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Apply LabelEncoder to each categorical column
for column in categorical_columns:
    train_data[column] = label_encoder.fit_transform(train_data[column])

# Display the first few rows of the updated DataFrame
train_data.head()

,EngineVersion,AppVersion,AvSigVersion,RtpStateBitfield,IsSxsPassiveMode,AVProductsInstalled,AVProductsEnabled,HasTpm,CityIdentifier,GeoNameIdentifier,LocaleEnglishNameIdentifier,Platform,Processor,OsVer,OsBuild,OsSuite,OsPlatformSubRelease,OsBuildLab,SkuEdition,IsProtected,SMode,SmartScreen,Firewall,Census_MDC2FormFactor,Census_DeviceFamily,Census_ProcessorCoreCount,Census_ProcessorModelIdentifier,Census_PrimaryDiskTotalCapacity,Census_PrimaryDiskTypeName,Census_SystemVolumeTotalCapacity,Census_HasOpticalDiskDrive,Census_TotalPhysicalRAM,Census_ChassisTypeName,Census_InternalPrimaryDiagonalDisplaySizeInInches,Census_InternalPrimaryDisplayResolutionHorizontal,Census_InternalPrimaryDisplayResolutionVertical,Census_PowerPlatformRoleName,Census_InternalBatteryNumberOfCharges,Census_OSVersion,Census_OSArchitecture,Census_OSBranch,Census_OSBuildNumber,Census_OSBuildRevision,Census_OSEdition,Census_OSSkuName,Census_OSInstallTypeName,Census_OSInstallLanguageIdentifier,Census_OSWUAutoUpdateOptionsName,Census_ActivationChannel,Census_FlightRing,Census_ThresholdOptIn,Census_IsSecureBootEnabled,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,HasDetections
0,51,48,5776,7.0,0.0,1.0,1.0,1.0,128035.0,35.0,171.0,0,1,0,17134.0,256.0,4,222,6,1.0,0.0,8,1.0,2,0,4.0,2341.0,476940.0,0,299451.0,0.0,4096.0,11,18.9,1440.0,900.0,1,4.294967e+09,249,0,5,17134.0,165.0,11,12,6,26.0,5,2,3,0.00029,0.0,0.0,0.0,0.0,0.0,0.0
1,41,23,4284,7.0,0.0,1.0,1.0,1.0,1482.0,119.0,64.0,0,1,0,17134.0,256.0,4,222,6,1.0,0.0,8,1.0,7,0,4.0,2405.0,476940.0,0,102385.0,0.0,4096.0,22,13.9,1366.0,768.0,3,1.000000e+00,246,0,5,17134.0,1.0,11,12,2,8.0,5,2,1,0.00029,0.0,0.0,0.0,0.0,0.0,0.0
2,51,48,5647,7.0,0.0,1.0,1.0,1.0,153579.0,64.0,49.0,0,1,0,17134.0,768.0,4,222,4,1.0,0.0,8,1.0,2,0,4.0,1972.0,114473.0,1,113907.0,0.0,4096.0,11,21.5,1920.0,1080.0,1,4.294967e+09,249,0,5,17134.0,165.0,1,1,6,7.0,2,1,3,0.00029,0.0,0.0,0.0,0.0,0.0,0.0
3,51,48,5698,7.0,0.0,1.0,1.0,1.0,20710.0,117.0,115.0,0,1,0,17134.0,256.0,4,222,6,1.0,0.0,3,1.0,2,0,4.0,2273.0,238475.0,2,227116.0,0.0,4096.0,20,18.5,1366.0,768.0,1,4.294967e+09,252,0,5,17134.0,228.0,11,12,6,17.0,2,1,3,0.00029,0.0,0.0,0.0,0.0,0.0,1.0
4,51,48,5657,7.0,0.0,1.0,1.0,1.0,37376.0,277.0,75.0,0,1,0,17134.0,768.0,4,222,4,1.0,0.0,8,1.0,7,0,4.0,2500.0,476940.0,0,101900.0,0.0,6144.0,24,14.0,1366.0,768.0,3,0.000000e+00,251,0,5,17134.0,191.0,1,1,7,8.0,2,2,3,0.00000,0.0,0.0,0.0,0.0,0.0,1.0


In [16]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 57 columns):
 #   Column                                             Non-Null Count   Dtype  
---  ------                                             --------------   -----  
 0   EngineVersion                                      500000 non-null  int64  
 1   AppVersion                                         500000 non-null  int64  
 2   AvSigVersion                                       500000 non-null  int64  
 3   RtpStateBitfield                                   500000 non-null  float64
 4   IsSxsPassiveMode                                   500000 non-null  float64
 5   AVProductsInstalled                                500000 non-null  float64
 6   AVProductsEnabled                                  500000 non-null  float64
 7   HasTpm                                             500000 non-null  float64
 8   CityIdentifier                                     500000 non-null  float6

In [17]:
# train_data.head

# Handle outliers

In [18]:


# Function to remove outliers using the IQR method
def remove_outliers_iqr(df):
    numeric_cols = df.select_dtypes(include=['number']).columns

    for col in numeric_cols:
        Q1 = df[col].quantile(0.25)  # First quartile (25%)
        Q3 = df[col].quantile(0.75)  # Third quartile (75%)
        IQR = Q3 - Q1  # Interquartile range (IQR)

        # Define the bounds for non-outliers
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Filter the data to keep only non-outliers
        df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]

    return df

# Apply the function to remove outliers from train_data
train_data_cleaned = remove_outliers_iqr(train_data)

# Apply the function to remove outliers from test_data (if available)
test_data_cleaned = remove_outliers_iqr(test_data)  # Replace 'test_data' with your actual test DataFrame variable name


In [19]:
train_data_cleaned.shape

(23366, 57)

# Handle skewness

In [20]:
nums_cols = train_data.select_dtypes(include=['int8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64']).columns
# Calculate skewness for numeric columns
skewness_df = train_data[nums_cols].skew().to_frame().rename(columns={0: "Feature Skewness"})

# Display the skewness DataFrame
print(skewness_df)

                                                   Feature Skewness
EngineVersion                                             -4.706520
AppVersion                                                -0.200658
AvSigVersion                                              -3.455019
RtpStateBitfield                                          -6.485635
IsSxsPassiveMode                                           7.407307
AVProductsInstalled                                        1.344681
AVProductsEnabled                                          5.203991
HasTpm                                                    -8.895397
CityIdentifier                                            -0.021201
GeoNameIdentifier                                         -0.219942
LocaleEnglishNameIdentifier                                0.573585
Platform                                                   5.459495
Processor                                                  2.840574
OsVer                                           

In [21]:
x_train = train_data.drop('HasDetections', axis=1)  # Features
y_train = train_data['HasDetections']  # Target

In [22]:

# Save the column names of the original x_train DataFrame
column_names = x_train.columns  # Assuming x_train_df was the original DataFrame

# Standard scaling process
scaler = StandardScaler()

# Fit and transform only the numeric columns
x_train_scaled = scaler.fit_transform(x_train)

# Convert the scaled NumPy array back to a DataFrame using the saved column names
x_train_scaled_df = pd.DataFrame(x_train_scaled, columns=column_names)

# Now you can use .head() to see the top rows of the scaled DataFrame
x_train_scaled_df.head()
x_train = x_train_scaled_df

# Feature selection

In [23]:
# Feature Selection using Chi-Square
# Ensure all data is non-negative before applying chi2
x_train_non_negative = x_train.abs()
chi2_selector = SelectKBest(score_func=chi2, k='all')

# Convert y_train to integers if it's continuous
y_train = y_train.astype(int)

x_train_chi2 = chi2_selector.fit_transform(x_train_non_negative, y_train)
chi2_scores = chi2_selector.scores_
chi2_features = x_train.columns[chi2_selector.get_support()]

In [24]:
# Convert chi2 scores to DataFrame for easy handling
chi2_scores_df = pd.DataFrame({'Feature': x_train.columns, 'Chi2_Score': chi2_scores})
top_chi2_features = chi2_scores_df.nlargest(6, 'Chi2_Score')['Feature'].tolist()

In [25]:
# Feature Selection using ANOVA
anova_selector = SelectKBest(score_func=f_classif, k='all')
x_train_anova = anova_selector.fit_transform(x_train, y_train)
anova_scores = anova_selector.scores_
anova_features = x_train.columns[anova_selector.get_support()]

In [26]:
# Convert anova scores to DataFrame for easy handling
anova_scores_df = pd.DataFrame({'Feature': x_train.columns, 'ANOVA_Score': anova_scores})
top_anova_features = anova_scores_df.nlargest(14, 'ANOVA_Score')['Feature'].tolist()

In [27]:
from sklearn.feature_selection import SelectKBest, mutual_info_regression
# Feature Selection using Mutual Information for Regression
mi_selector = SelectKBest(score_func=mutual_info_regression, k='all')
x_train_mi = mi_selector.fit_transform(x_train, y_train)
mi_scores = mi_selector.scores_
mi_features = x_train.columns[mi_selector.get_support()]

In [28]:
# Convert mutual information scores to DataFrame for easy handling
mi_scores_df = pd.DataFrame({'Feature': x_train.columns, 'MI_Score': mi_scores})
top_mi_features = mi_scores_df.nlargest(10, 'MI_Score')['Feature'].tolist()

In [29]:
# Combine top features from all methods (union)
combined_features = set(top_chi2_features) | set(top_anova_features) | set(top_mi_features)

# Output the result
print("Combined Top Features from All Methods:")
print(combined_features)
print("Total number of unique combined features:", len(combined_features))

Combined Top Features from All Methods:
{'Census_PrimaryDiskTotalCapacity', 'Census_ProcessorCoreCount', 'Census_PowerPlatformRoleName', 'IsProtected', 'OsBuildLab', 'Census_OSArchitecture', 'AppVersion', 'EngineVersion', 'Census_SystemVolumeTotalCapacity', 'Census_IsAlwaysOnAlwaysConnectedCapable', 'Processor', 'SmartScreen', 'Census_ProcessorModelIdentifier', 'AVProductsInstalled', 'Census_IsTouchEnabled', 'AVProductsEnabled', 'Census_TotalPhysicalRAM', 'AvSigVersion', 'Wdft_IsGamer'}
Total number of unique combined features: 19


In [30]:
# Filter the dataset
combined_features_list = list(combined_features)
X_train = train_data[combined_features_list]
X_test = test_data[combined_features_list]
print("Shape of Filtered Training Dataset:", X_train.shape)
print("Shape of Filtered Test Dataset:", X_test.shape)

Shape of Filtered Training Dataset: (500000, 19)
Shape of Filtered Test Dataset: (500000, 19)


In [31]:
print(X_train.columns)

Index(['Census_PrimaryDiskTotalCapacity', 'Census_ProcessorCoreCount',
       'Census_PowerPlatformRoleName', 'IsProtected', 'OsBuildLab',
       'Census_OSArchitecture', 'AppVersion', 'EngineVersion',
       'Census_SystemVolumeTotalCapacity',
       'Census_IsAlwaysOnAlwaysConnectedCapable', 'Processor', 'SmartScreen',
       'Census_ProcessorModelIdentifier', 'AVProductsInstalled',
       'Census_IsTouchEnabled', 'AVProductsEnabled', 'Census_TotalPhysicalRAM',
       'AvSigVersion', 'Wdft_IsGamer'],
      dtype='object')


In [32]:
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
X__Train, X__val, y__Train, y__val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [33]:
# !pkill ngrok

In [34]:
# Step 1: Set up ngrok for MLflow UI
# port = 5000
# ngrok_tunnel = ngrok.connect(port, "http")
# print(f'ngrok tunnel URL: {ngrok_tunnel.public_url}')
# mlflow.set_tracking_uri(ngrok_tunnel.public_url)

# Random forest

In [35]:
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
import pandas as pd

# Define and train the Random Forest model
rf = RandomForestClassifier(n_estimators=100, max_depth=20, min_samples_split=10, min_samples_leaf=5, random_state=42)
rf.fit(X__Train, y__Train)

# Predictions
y_train_pred_rf = rf.predict(X__Train)
y_val_pred_rf = rf.predict(X__val)
y_train_prob_rf = rf.predict_proba(X__Train)[:, 1]

# Performance metrics
accuracy_train = accuracy_score(y__Train, y_train_pred_rf)
accuracy_val = accuracy_score(y__val, y_val_pred_rf)
roc_auc_train = roc_auc_score(y__Train, y_train_prob_rf)
f1_train = f1_score(y__Train, y_train_pred_rf)

print(f"Random Forest Classifier:")
print(f"Training Accuracy: {accuracy_train:.4f}")
print(f"Validation Accuracy: {accuracy_val:.4f}")
print(f"Training ROC AUC: {roc_auc_train:.4f}")
print(f"Training F1 Score: {f1_train:.4f}")

Random Forest Classifier:
Training Accuracy: 0.7055
Validation Accuracy: 0.6360
Training ROC AUC: 0.7889
Training F1 Score: 0.7066


In [36]:
import os

# Create a directory for MLflow logs
mlflow_dir = "/content/mlflow"
os.makedirs(mlflow_dir, exist_ok=True)

# Set the tracking URI to the new directory
# Set the tracking URI to log to the local file
# mlflow.set_tracking_uri("file:./mlruns")

# Create a new experiment
mlflow.set_experiment("Microsoft_Malware_Experiment")

2024/09/25 12:40:22 INFO mlflow.tracking.fluent: Experiment with name 'Microsoft_Malware_Experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///content/mlruns/491042851560132415', creation_time=1727268022261, experiment_id='491042851560132415', last_update_time=1727268022261, lifecycle_stage='active', name='Microsoft_Malware_Experiment', tags={}>

In [37]:
# Create an example input with actual values
input_example = pd.DataFrame([X__Train.iloc[0]], columns=X__Train.columns)

# Start MLflow run
with mlflow.start_run():
    # Log the model with input_example
    mlflow.sklearn.log_model(rf, "random_forest_model", input_example=input_example)
    mlflow.log_metric("accuracy_train", accuracy_train)
    mlflow.log_metric("accuracy_val", accuracy_val)
    mlflow.log_metric("roc_auc_train", roc_auc_train)
    mlflow.log_metric("f1_train", f1_train)
    mlflow.log_param("n_estimators", 100)
    mlflow.log_param("max_depth", 20)
    mlflow.log_param("min_samples_split", 10)
    mlflow.log_param("min_samples_leaf", 5)

print("Model and metrics logged to MLflow")


/usr/local/lib/python3.10/dist-packages/mlflow/types/utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


Model and metrics logged to MLflow


In [40]:
# Step 5: Set the tracking URI for MLflow
mlflow.set_tracking_uri("file:///content/mlruns")

# Step 6: Kill existing ngrok tunnels (if any)
ngrok.kill()  # Terminate any existing tunnels

# Step 7: Start the MLflow UI
!mlflow ui --port 5000 --host 0.0.0.0 &

# Step 8: Start an ngrok tunnel for port 5000
mlflow_url = ngrok.connect(5000, "http")  # Create the ngrok tunnel
print(f"MLflow Tracking UI available at: {mlflow_url}")  # Print the public URL


[2024-09-25 12:44:25 +0000] [5740] [INFO] Starting gunicorn 23.0.0
[2024-09-25 12:44:25 +0000] [5740] [INFO] Listening at: http://0.0.0.0:5000 (5740)
[2024-09-25 12:44:25 +0000] [5740] [INFO] Using worker: sync
[2024-09-25 12:44:25 +0000] [5741] [INFO] Booting worker with pid: 5741
[2024-09-25 12:44:25 +0000] [5742] [INFO] Booting worker with pid: 5742
[2024-09-25 12:44:25 +0000] [5743] [INFO] Booting worker with pid: 5743
[2024-09-25 12:44:25 +0000] [5744] [INFO] Booting worker with pid: 5744
[2024-09-25 12:45:13 +0000] [5740] [INFO] Handling signal: int
[2024-09-25 12:45:13 +0000] [5741] [INFO] Worker exiting (pid: 5741)
[2024-09-25 12:45:13 +0000] [5742] [INFO] Worker exiting (pid: 5742)
[2024-09-25 12:45:13 +0000] [5743] [INFO] Worker exiting (pid: 5743)
[2024-09-25 12:45:13 +0000] [5744] [INFO] Worker exiting (pid: 5744)
[2024-09-25 12:45:15 +0000] [5740] [INFO] Shutting down: Master
MLflow Tracking UI available at: NgrokTunnel: "https://73e8-35-236-161-206.ngrok-free.app" -> "htt

# XGBOOST

In [41]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
import mlflow
import mlflow.sklearn

# Define and train the XGBoost model
xgb_model = xgb.XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, subsample=0.8, colsample_bytree=0.8, random_state=42)
y__Train = y__Train.replace(-1, 1)  # Replace -1 with 1 if required
xgb_model.fit(X__Train, y__Train)

# Predictions and probabilities
y_train_pred_xgb = xgb_model.predict(X__Train)
y_val_pred_xgb = xgb_model.predict(X__val)
y_train_prob_xgb = xgb_model.predict_proba(X__Train)[:, 1]

# Performance metrics
accuracy_train = accuracy_score(y__Train, y_train_pred_xgb)
accuracy_val = accuracy_score(y__val, y_val_pred_xgb)
roc_auc_train = roc_auc_score(y__Train, y_train_prob_xgb)
f1_train = f1_score(y__Train, y_train_pred_xgb)

print(f"XGBoost Classifier:")
print(f"Training Accuracy: {accuracy_train:.4f}")
print(f"Validation Accuracy: {accuracy_val:.4f}")
print(f"Training ROC AUC: {roc_auc_train:.4f}")
print(f"Training F1 Score: {f1_train:.4f}")

# Example input data for MLflow logging
input_example_xgb = pd.DataFrame(X__Train.columns).T

# Log model and metrics to MLflow
with mlflow.start_run():
    mlflow.xgboost.log_model(xgb_model, "xgboost_model", input_example=input_example_xgb)
    mlflow.log_metric("accuracy_train", accuracy_train)
    mlflow.log_metric("accuracy_val", accuracy_val)
    mlflow.log_metric("roc_auc_train", roc_auc_train)
    mlflow.log_metric("f1_train", f1_train)
    mlflow.log_param("n_estimators", 100)
    mlflow.log_param("learning_rate", 0.1)
    mlflow.log_param("max_depth", 5)
    mlflow.log_param("subsample", 0.8)
    mlflow.log_param("colsample_bytree", 0.8)

# print(f"Model and metrics logged to MLflow at: {ngrok_tunnel.public_url}")


2024/09/25 12:45:41 WARNING mlflow.models.signature: Failed to infer the model signature from the input example. Reason: ValueError('DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, the experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:0: object, 1: object, 2: object, 3: object, 4: object, 5: object, 6: object, 7: object, 8: object, 9: object, 10: object, 11: object, 12: object, 13: object, 14: object, 15: object, 16: object, 17: object, 18: object'). To see the full traceback, set the logging level to DEBUG via `logging.getLogger("mlflow").setLevel(logging.DEBUG)`. To disable automatic signature inference, set `signature` to `False` in your `log_model` or `save_model` call.


XGBoost Classifier:
Training Accuracy: 0.6369
Validation Accuracy: 0.6328
Training ROC AUC: 0.6951
Training F1 Score: 0.6353


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [12:45:41] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
2024/09/25 12:45:46 WARNING mlflow.utils.requirements_utils: Failed to run predict on input_example, dependencies introduced in predict are not captured.
ValueError('DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, the experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:0: object, 1: object, 2: object, 3: object, 4: object, 5: object, 6: object, 7: object, 8: object, 9: object, 10: object, 11: object, 12: object, 13: object, 14: object, 15: object, 16: object, 17: object, 18: object')Traceback (most recent call last):


  File "/usr/local/lib/python3.10/dist-packages/mlflow/utils/_capture_modules.py", line 

2024/09/25 12:45:48 WARNING mlflow.models.model: Failed to validate serving input example {
  "dataframe_split": {
    "data": [
      [
        "Census_PrimaryDiskTotalCapacity",
        "Census_ProcessorCoreCount",
        "Census_PowerPlatformRoleName",
        "IsProtected",
        "OsBuildLab",
        "Census_OSArchitecture",
        "AppVersion",
        "EngineVersion",
        "Census_SystemVolumeTotalCapacity",
        "Census_IsAlwaysOnAlwaysConnectedCapable",
        "Processor",
        "SmartScreen",
        "Census_ProcessorModelIdentifier",
        "AVProductsInstalled",
        "Census_IsTouchEnabled",
        "AVProductsEnabled",
        "Census_TotalPhysicalRAM",
        "AvSigVersion",
        "Wdft_IsGamer"
      ]
    ]
  }
}. Alternatively, you can avoid passing input example and pass model signature instead when logging the model. To ensure the input example is valid prior to serving, please try calling `mlflow.models.validate_serving_input` on the model uri an

NameError: name 'ngrok_tunnel' is not defined

# LIGHT

In [42]:
import lightgbm as lgb
import mlflow
import mlflow.lightgbm
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score

# Define and train the LightGBM model
lgb_model = lgb.LGBMClassifier(n_estimators=100, learning_rate=0.1, num_leaves=50, subsample=0.8, colsample_bytree=0.8, random_state=42)
lgb_model.fit(X__Train, y__Train)

# Predictions and probabilities
y_train_pred_lgb = lgb_model.predict(X__Train)
y_val_pred_lgb = lgb_model.predict(X__val)
y_train_prob_lgb = lgb_model.predict_proba(X__Train)[:, 1]

# Performance metrics
accuracy_train = accuracy_score(y__Train, y_train_pred_lgb)
accuracy_val = accuracy_score(y__val, y_val_pred_lgb)
roc_auc_train = roc_auc_score(y__Train, y_train_prob_lgb)
f1_train = f1_score(y__Train, y_train_pred_lgb)

print(f"LightGBM Classifier:")
print(f"Training Accuracy: {accuracy_train:.4f}")
print(f"Validation Accuracy: {accuracy_val:.4f}")
print(f"Training ROC AUC: {roc_auc_train:.4f}")
print(f"Training F1 Score: {f1_train:.4f}")

# Example input data for MLflow logging
input_example_lgb = pd.DataFrame(X__Train.columns).T

# Log model and metrics to MLflow
with mlflow.start_run():
    mlflow.lightgbm.log_model(lgb_model, "lightgbm_model", input_example=input_example_lgb)
    mlflow.log_metric("accuracy_train", accuracy_train)
    mlflow.log_metric("accuracy_val", accuracy_val)
    mlflow.log_metric("roc_auc_train", roc_auc_train)
    mlflow.log_metric("f1_train", f1_train)
    mlflow.log_param("n_estimators", 100)
    mlflow.log_param("learning_rate", 0.1)
    mlflow.log_param("num_leaves", 50)
    mlflow.log_param("subsample", 0.8)
    mlflow.log_param("colsample_bytree", 0.8)

# print(f"Model and metrics logged to MLflow at: {ngrok_tunnel.public_url}")

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


[LightGBM] [Info] Number of positive: 200476, number of negative: 199524
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.124760 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1375
[LightGBM] [Info] Number of data points in the train set: 400000, number of used features: 19
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.501190 -> initscore=0.004760
[LightGBM] [Info] Start training from score 0.004760
LightGBM Classifier:
Training Accuracy: 0.6440
Validation Accuracy: 0.6360
Training ROC AUC: 0.7040
Training F1 Score: 0.6418


2024/09/25 12:47:12 WARNING mlflow.models.signature: Failed to infer the model signature from the input example. Reason: ValueError('pandas dtypes must be int, float or bool.\nFields with bad pandas dtypes: 0: object, 1: object, 2: object, 3: object, 4: object, 5: object, 6: object, 7: object, 8: object, 9: object, 10: object, 11: object, 12: object, 13: object, 14: object, 15: object, 16: object, 17: object, 18: object'). To see the full traceback, set the logging level to DEBUG via `logging.getLogger("mlflow").setLevel(logging.DEBUG)`. To disable automatic signature inference, set `signature` to `False` in your `log_model` or `save_model` call.
2024/09/25 12:47:20 WARNING mlflow.utils.requirements_utils: Failed to run predict on input_example, dependencies introduced in predict are not captured.
ValueError('pandas dtypes must be int, float or bool.\nFields with bad pandas dtypes: 0: object, 1: object, 2: object, 3: object, 4: object, 5: object, 6: object, 7: object, 8: object, 9: obj

2024/09/25 12:47:21 WARNING mlflow.models.model: Failed to validate serving input example {
  "dataframe_split": {
    "data": [
      [
        "Census_PrimaryDiskTotalCapacity",
        "Census_ProcessorCoreCount",
        "Census_PowerPlatformRoleName",
        "IsProtected",
        "OsBuildLab",
        "Census_OSArchitecture",
        "AppVersion",
        "EngineVersion",
        "Census_SystemVolumeTotalCapacity",
        "Census_IsAlwaysOnAlwaysConnectedCapable",
        "Processor",
        "SmartScreen",
        "Census_ProcessorModelIdentifier",
        "AVProductsInstalled",
        "Census_IsTouchEnabled",
        "AVProductsEnabled",
        "Census_TotalPhysicalRAM",
        "AvSigVersion",
        "Wdft_IsGamer"
      ]
    ]
  }
}. Alternatively, you can avoid passing input example and pass model signature instead when logging the model. To ensure the input example is valid prior to serving, please try calling `mlflow.models.validate_serving_input` on the model uri an

# ADABOOST

In [43]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
import mlflow
import mlflow.sklearn

# Define and train the AdaBoost model
ada = AdaBoostClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
ada.fit(X__Train, y__Train)

# Predictions and probabilities
y_train_pred_ada = ada.predict(X__Train)
y_val_pred_ada = ada.predict(X__val)
y_train_prob_ada = ada.predict_proba(X__Train)[:, 1]

# Performance metrics
accuracy_train_ada = accuracy_score(y__Train, y_train_pred_ada)
accuracy_val_ada = accuracy_score(y__val, y_val_pred_ada)
roc_auc_train_ada = roc_auc_score(y__Train, y_train_prob_ada)
f1_train_ada = f1_score(y__Train, y_train_pred_ada)

print(f"AdaBoost Classifier:")
print(f"Training Accuracy: {accuracy_train_ada:.4f}")
print(f"Validation Accuracy: {accuracy_val_ada:.4f}")
print(f"Training ROC AUC: {roc_auc_train_ada:.4f}")
print(f"Training F1 Score: {f1_train_ada:.4f}")

# Example input data for MLflow logging
input_example_ada = pd.DataFrame(X__Train.columns).T

# Log model and metrics to MLflow
with mlflow.start_run():
    mlflow.sklearn.log_model(ada, "adaboost_model", input_example=input_example_ada)
    mlflow.log_metric("accuracy_train", accuracy_train_ada)
    mlflow.log_metric("accuracy_val", accuracy_val_ada)
    mlflow.log_metric("roc_auc_train", roc_auc_train_ada)
    mlflow.log_metric("f1_train", f1_train_ada)
    mlflow.log_param("n_estimators", 100)
    mlflow.log_param("learning_rate", 0.1)

# print(f"Model and metrics logged to MLflow at: {ngrok_tunnel.public_url}")


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but AdaBoostClassifier was fitted with feature names
  warnings.warn(
2024/09/25 12:48:16 WARNING mlflow.models.signature: Failed to infer the model signature from the input example. Reason: ValueError("could not convert string to float: 'Census_PrimaryDiskTotalCapacity'"). To see the full traceback, set the logging level to DEBUG via `logging.getLogger("mlflow").setLevel(logging.DEBUG)`. To disable automatic signature inference, set `signature` to `False` in your `log_model` or `save_model` call.


AdaBoost Classifier:
Training Accuracy: 0.6184
Validation Accuracy: 0.6193
Training ROC AUC: 0.6697
Training F1 Score: 0.6380


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but AdaBoostClassifier was fitted with feature names
  warnings.warn(
2024/09/25 12:48:22 WARNING mlflow.models.model: Failed to validate serving input example {
  "dataframe_split": {
    "data": [
      [
        "Census_PrimaryDiskTotalCapacity",
        "Census_ProcessorCoreCount",
        "Census_PowerPlatformRoleName",
        "IsProtected",
        "OsBuildLab",
        "Census_OSArchitecture",
        "AppVersion",
        "EngineVersion",
        "Census_SystemVolumeTotalCapacity",
        "Census_IsAlwaysOnAlwaysConnectedCapable",
        "Processor",
        "SmartScreen",
        "Census_ProcessorModelIdentifier",
        "AVProductsInstalled",
        "Census_IsTouchEnabled",
        "AVProductsEnabled",
        "Census_TotalPhysicalRAM",
        "AvSigVersion",
        "Wdft_IsGamer"
      ]
    ]
  }
}. Alternatively, you can avoid passing input example and pass

# Logestic reg

In [44]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
import mlflow
import mlflow.sklearn
import pandas as pd

# Define and train the Logistic Regression model
log_reg = LogisticRegression(C=1.0, solver='lbfgs', max_iter=100, random_state=42)
log_reg.fit(X__Train, y__Train)

# Predictions and probabilities
y_train_pred_log_reg = log_reg.predict(X__Train)
y_val_pred_log_reg = log_reg.predict(X__val)
y_train_prob_log_reg = log_reg.predict_proba(X__Train)[:, 1]

# Performance metrics
accuracy_train_log_reg = accuracy_score(y__Train, y_train_pred_log_reg)
accuracy_val_log_reg = accuracy_score(y__val, y_val_pred_log_reg)
roc_auc_train_log_reg = roc_auc_score(y__Train, y_train_prob_log_reg)
f1_train_log_reg = f1_score(y__Train, y_train_pred_log_reg)

print(f"Logistic Regression:")
print(f"Training Accuracy: {accuracy_train_log_reg:.4f}")
print(f"Validation Accuracy: {accuracy_val_log_reg:.4f}")
print(f"Training ROC AUC: {roc_auc_train_log_reg:.4f}")
print(f"Training F1 Score: {f1_train_log_reg:.4f}")

# Example input data for MLflow logging
input_example_log_reg = pd.DataFrame(X__Train.columns).T

# Log model and metrics to MLflow
with mlflow.start_run():
    mlflow.sklearn.log_model(log_reg, "logistic_regression_model", input_example=input_example_log_reg)
    mlflow.log_metric("accuracy_train", accuracy_train_log_reg)
    mlflow.log_metric("accuracy_val", accuracy_val_log_reg)
    mlflow.log_metric("roc_auc_train", roc_auc_train_log_reg)
    mlflow.log_metric("f1_train", f1_train_log_reg)
    mlflow.log_param("C", 1.0)
    mlflow.log_param("solver", 'lbfgs')
    mlflow.log_param("max_iter", 100)

print(f"Model and metrics logged to MLflow.")


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
2024/09/25 12:48:38 WARNING mlflow.models.signature: Failed to infer the model signature from the input example. Reason: ValueError("could not convert string to float: 'Census_PrimaryDiskTotalCapacity'"). To see the full traceback, set the logging level to DEBUG via `logging.getLogger("mlflow").setLevel(log

Logistic Regression:
Training Accuracy: 0.5349
Validation Accuracy: 0.5362
Training ROC AUC: 0.5519
Training F1 Score: 0.5141


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
2024/09/25 12:48:49 WARNING mlflow.models.model: Failed to validate serving input example {
  "dataframe_split": {
    "data": [
      [
        "Census_PrimaryDiskTotalCapacity",
        "Census_ProcessorCoreCount",
        "Census_PowerPlatformRoleName",
        "IsProtected",
        "OsBuildLab",
        "Census_OSArchitecture",
        "AppVersion",
        "EngineVersion",
        "Census_SystemVolumeTotalCapacity",
        "Census_IsAlwaysOnAlwaysConnectedCapable",
        "Processor",
        "SmartScreen",
        "Census_ProcessorModelIdentifier",
        "AVProductsInstalled",
        "Census_IsTouchEnabled",
        "AVProductsEnabled",
        "Census_TotalPhysicalRAM",
        "AvSigVersion",
        "Wdft_IsGamer"
      ]
    ]
  }
}. Alternatively, you can avoid passing input example and pass

Model and metrics logged to MLflow.
